In [1]:
import json
import xml.etree.ElementTree as ET
import random

# 打开JSON文件
with open('./scenario_rep/scenario_parameters.json', 'r') as file:
    # 解析JSON数据
    scenario = json.load(file)

# 使用数据
# 例如，访问JSON中的特定字段
print(scenario["weather_density"],'\n',scenario["time"],'\n',\
      scenario["road_type"],'\n',scenario["actors"])
weather_density_ = scenario["weather_density"]
time_ = scenario["time"]
road_type_ = scenario["road_type"]
actors_ = scenario["actors"]
try:
    temporary_changes_ = scenario["temporary changes"]
except:
    pass


{'rainy': 1.0} 
 ['nighttime'] 
 ['intersection'] 
 {'ego_vehicle': {'type': ['car'], 'lane_relation': ['left lane'], 'behavior': ['turn left']}, 'npc_actor': {'type': ['car'], 'initial_position_relative_to_ego': ['right'], 'initial_lane_relative_to_ego': ['right lane'], 'oracle_of_npc_actor': ['accelerate', 'change lane to left', 'overtake', 'stop']}}


In [2]:
def _indent(elem, level=0):
    # xml文件自动换行函数！
    i = "\n" + level*"\t"
    if len(elem):
        if not elem.text or not elem.text.strip():
            elem.text = i + "\t"
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
        for elem in elem:
            _indent(elem, level+1)
        if not elem.tail or not elem.tail.strip():
            elem.tail = i
    else:
        if level and (not elem.tail or not elem.tail.strip()):
            elem.tail = i

In [3]:
def write_xosc_start(maneuver):
    event = ET.SubElement(maneuver, 'Event', {'name': 'AdversaryStart0', 'priority': 'overwrite'})
    action = ET.SubElement(event, 'Action', {'name': 'EVENT0'}) #action
    private_action = ET.SubElement(action, 'PrivateAction')
    LongitudianlAction = ET.SubElement(private_action, 'LongitudinalAction')
    SpeedAction = ET.SubElement(LongitudianlAction, 'SpeedAction')
    SpeedActionDynamics = ET.SubElement(SpeedAction, 'SpeedActionDynamics',\
                                        {'dynamicsShape':'step',\
                                        'value':'50','dynamicsDimension':'distance'})
    SpeedActionTarget = ET.SubElement(SpeedAction,'SpeedActionTarget')
    AbsoluteTargetSpeed = ET.SubElement(SpeedActionTarget,'AbsoluteTargetSpeed',{'value':\
                                                                                    '10'})
    
    start_trigger = ET.SubElement(event,'StartTrigger') #st
    condition_group = ET.SubElement(start_trigger, 'ConditionGroup')
    condition = ET.SubElement(condition_group, 'Condition', {'name': 'StartCondition', 'delay': '0',\
                                                                'conditionEdge': 'rising'})
    by_entity_condition = ET.SubElement(condition, 'ByEntityCondition')
    triggering_entities = ET.SubElement(by_entity_condition, 'TriggeringEntities', \
                                        {'triggeringEntitiesRule': 'any'})
    entity_ref_hero = ET.SubElement(triggering_entities, 'EntityRef', {'entityRef': 'hero'})
    entity_condition = ET.SubElement(by_entity_condition, 'EntityCondition')
    relative_distance_condition = ET.SubElement(entity_condition, 'RelativeDistanceCondition', \
        {
        'entityRef': 'adversary',
        'relativeDistanceType': 'cartesianDistance',
        'value': '400.0',
        'freespace': 'false',
        'rule': 'lessThan'
    })

def write_xosc_accelerate(maneuver,EVENT_ID):
    event = ET.SubElement(maneuver, 'Event', {'name': 'AdversaryAccelerates'+str(EVENT_ID), 'priority': 'overwrite'})
    action = ET.SubElement(event, 'Action', {'name': 'EVENT'+str(EVENT_ID)}) #action
    private_action = ET.SubElement(action, 'PrivateAction')
    LongitudianlAction = ET.SubElement(private_action, 'LongitudinalAction')
    SpeedAction = ET.SubElement(LongitudianlAction, 'SpeedAction')
    SpeedActionDynamics = ET.SubElement(SpeedAction, 'SpeedActionDynamics',\
                                        {'dynamicsShape':'linear',\
                                        'value':'500','dynamicsDimension':'distance'})
    SpeedActionTarget = ET.SubElement(SpeedAction,'SpeedActionTarget')
    AbsoluteTargetSpeed = ET.SubElement(SpeedActionTarget,'AbsoluteTargetSpeed',{'value':\
                                                                                    '20'})
    
    start_trigger = ET.SubElement(event, 'StartTrigger')
    condition_group = ET.SubElement(start_trigger, 'ConditionGroup')
    condition = ET.SubElement(condition_group, 'Condition', {
        'name': 'AfterAdversaryAction',
        'delay': '0',
        'conditionEdge': 'rising'
    })
    by_value_condition = ET.SubElement(condition, 'ByValueCondition')
    storyboard_element_state_condition = ET.SubElement(by_value_condition, \
                                                        'StoryboardElementStateCondition', {
        'storyboardElementType': 'action',
        'storyboardElementRef': 'EVENT'+str(EVENT_ID-1),
        'state': 'completeState'
    })

def write_xosc_change_lane(maneuver,change_value,EVENT_ID):
    event = ET.SubElement(maneuver, 'Event', {'name': 'AdversaryChangesLane'+str(EVENT_ID), 'priority': 'overwrite'})
    action = ET.SubElement(event, 'Action', {'name': 'EVENT'+str(EVENT_ID)})
    private_action = ET.SubElement(action, 'PrivateAction')
    lateral_action = ET.SubElement(private_action, 'LateralAction')
    lane_change_action = ET.SubElement(lateral_action, 'LaneChangeAction')
    lane_change_action_dynamics = ET.SubElement(lane_change_action, 'LaneChangeActionDynamics', {
        'dynamicsShape': 'linear',
        'value': '12',
        'dynamicsDimension': 'distance'
    })
    lane_change_target = ET.SubElement(lane_change_action, 'LaneChangeTarget')
    relative_target_lane = ET.SubElement(lane_change_target, 'RelativeTargetLane', {
        'entityRef': 'adversary',
        'value': change_value #右-1，左1
    })

    start_trigger = ET.SubElement(event, 'StartTrigger')
    condition_group = ET.SubElement(start_trigger, 'ConditionGroup')
    condition = ET.SubElement(condition_group, 'Condition', {
        'name': 'AfterAdversaryAction',
        'delay': '0',
        'conditionEdge': 'rising'
    })
    by_value_condition = ET.SubElement(condition, 'ByValueCondition')
    storyboard_element_state_condition = ET.SubElement(by_value_condition, \
                                                        'StoryboardElementStateCondition', {
        'storyboardElementType': 'action',
        'storyboardElementRef': 'EVENT'+str(EVENT_ID-1),
        'state': 'completeState'
    })

def write_xosc_stop(maneuver,EVENT_ID):
    event = ET.SubElement(maneuver, 'Event', {'name': 'AdversaryStop'+str(EVENT_ID), 'priority': 'overwrite'})
    action = ET.SubElement(event, 'Action', {'name': 'EVENT'+str(EVENT_ID)}) #action
    private_action = ET.SubElement(action, 'PrivateAction')
    LongitudianlAction = ET.SubElement(private_action, 'LongitudinalAction')
    SpeedAction = ET.SubElement(LongitudianlAction, 'SpeedAction')
    SpeedActionDynamics = ET.SubElement(SpeedAction, 'SpeedActionDynamics',\
                                        {'dynamicsShape':'linear',\
                                        'value':'10','dynamicsDimension':'distance'})
    SpeedActionTarget = ET.SubElement(SpeedAction,'SpeedActionTarget')
    AbsoluteTargetSpeed = ET.SubElement(SpeedActionTarget,'AbsoluteTargetSpeed',{'value':\
                                                                                    '0'})
    
    start_trigger = ET.SubElement(event, 'StartTrigger')
    condition_group = ET.SubElement(start_trigger, 'ConditionGroup')
    condition = ET.SubElement(condition_group, 'Condition', {
        'name': 'AfterAdversaryAction',
        'delay': '0',
        'conditionEdge': 'rising'
    })
    by_value_condition = ET.SubElement(condition, 'ByValueCondition')
    storyboard_element_state_condition = ET.SubElement(by_value_condition, \
                                                        'StoryboardElementStateCondition', {
        'storyboardElementType': 'action',
        'storyboardElementRef': 'EVENT'+str(EVENT_ID-1),
        'state': 'completeState'
    })
    
def write_xosc_const(maneuver,EVENT_ID):
    if 'bike' in actors_["npc_actor"]["type"]:
        AbsoluteTargetSpeed = str(1)
    else:
        AbsoluteTargetSpeed = str(10)
    event = ET.SubElement(maneuver, 'Event', {'name': 'ConstantSpeed'+str(EVENT_ID), 'priority': 'overwrite'})
    action = ET.SubElement(event, 'Action', {'name': 'EVENT'+str(EVENT_ID)}) #action
    private_action = ET.SubElement(action, 'PrivateAction')
    LongitudianlAction = ET.SubElement(private_action, 'LongitudinalAction')
    SpeedAction = ET.SubElement(LongitudianlAction, 'SpeedAction')
    SpeedActionDynamics = ET.SubElement(SpeedAction, 'SpeedActionDynamics',\
                                        {'dynamicsShape':'step',\
                                        'value':'2000','dynamicsDimension':'distance'})
    SpeedActionTarget = ET.SubElement(SpeedAction, 'SpeedActionTarget')
    AbsoluteTargetSpeed = ET.SubElement(SpeedActionTarget, 'AbsoluteTargetSpeed',{'value': AbsoluteTargetSpeed})
    
    start_trigger = ET.SubElement(event, 'StartTrigger')
    condition_group = ET.SubElement(start_trigger, 'ConditionGroup')
    condition = ET.SubElement(condition_group, 'Condition', {
        'name': 'AfterAdversaryAction',
        'delay': '0',
        'conditionEdge': 'rising'
    })
    by_entity_condition = ET.SubElement(condition, 'ByEntityCondition')
    triggering_entities = ET.SubElement(by_entity_condition, 'TriggeringEntities', \
                                        {'triggeringEntitiesRule': 'any'})
    entity_ref_hero = ET.SubElement(triggering_entities, 'EntityRef', {'entityRef': 'adversary'})
    entity_condition = ET.SubElement(by_entity_condition, 'EntityCondition')
    relative_distance_condition = ET.SubElement(entity_condition, 'SpeedCondition', \
        {
        'value': '30.0',
        'rule': 'lessThan'
    })

def write_xosc_navigation(maneuver,EVENT_ID,TargetLocation_npc):
    # 创建根元素
    event = ET.SubElement(maneuver, 'Event', attrib={'name': 'RouteCreation'+str(EVENT_ID), 'priority': 'overwrite'})
    action = ET.SubElement(event, 'Action', attrib={'name': 'EVENT'+str(EVENT_ID)})
    private_action = ET.SubElement(action, 'PrivateAction')
    routing_action = ET.SubElement(private_action, 'RoutingAction')
    acquire_position_action = ET.SubElement(routing_action, 'AcquirePositionAction')
    position = ET.SubElement(acquire_position_action, 'Position')
    world_position = ET.SubElement(position, 'WorldPosition', attrib=\
                                    {'x': TargetLocation_npc['x'], 'y': TargetLocation_npc['y'],\
                                     'z': TargetLocation_npc['z'], 'h': TargetLocation_npc['h']})
    start_trigger = ET.SubElement(event, 'StartTrigger')
    condition_group = ET.SubElement(start_trigger, 'ConditionGroup')
    condition = ET.SubElement(condition_group, 'Condition', attrib={'name': 'StartCondition', 'delay': '0', 'conditionEdge': 'rising'})
    by_value_condition = ET.SubElement(condition, 'ByValueCondition')
    simulation_time_condition = ET.SubElement(by_value_condition, 'SimulationTimeCondition', \
                                                attrib={'value': '30', 'rule': 'greaterThan'})

def write_xosc_harzard_dec(maneuver,EVENT_ID,TargetLocation_npc):
    # 创建根元素
    event = ET.SubElement(maneuver, 'Event', {'name': 'AdversaryStop'+str(EVENT_ID), 'priority': 'overwrite'})
    action = ET.SubElement(event, 'Action', {'name': 'EVENT'+str(EVENT_ID)}) #action
    private_action = ET.SubElement(action, 'PrivateAction')
    LongitudianlAction = ET.SubElement(private_action, 'LongitudinalAction')
    SpeedAction = ET.SubElement(LongitudianlAction, 'SpeedAction')
    SpeedActionDynamics = ET.SubElement(SpeedAction, 'SpeedActionDynamics',\
                                        {'dynamicsShape':'linear',\
                                        'value':'10','dynamicsDimension':'distance'})
    SpeedActionTarget = ET.SubElement(SpeedAction,'SpeedActionTarget')
    AbsoluteTargetSpeed = ET.SubElement(SpeedActionTarget,'AbsoluteTargetSpeed',{'value':\
                                                                                    '0'})
    
    start_trigger = ET.SubElement(event, 'StartTrigger')
    condition_group = ET.SubElement(start_trigger, 'ConditionGroup')
    condition = ET.SubElement(condition_group, 'Condition', {
        'name': 'AfterAdversaryAction',
        'delay': '0',
        'conditionEdge': 'rising'
    })
    by_value_condition = ET.SubElement(condition, 'ByValueCondition')
    storyboard_element_state_condition = ET.SubElement(by_value_condition, \
                                                        'StoryboardElementStateCondition', {
        'storyboardElementType': 'action',
        'storyboardElementRef': 'EVENT'+str(EVENT_ID-1),
        'state': 'completeState'
    })


In [4]:
def events_of_npc_define(events,root,TargetLocation_npc):
    EVENT_ID = 0
    for maneuver_group in root.findall(".//ManeuverGroup"):
        for maneuver in maneuver_group.findall("Maneuver"):
            write_xosc_start(maneuver)
        EVENT_ID += 1
    
    for event in events:
        if event == 'accelerate':
            for maneuver_group in root.findall(".//ManeuverGroup"):
                for maneuver in maneuver_group.findall("Maneuver"):
                    write_xosc_accelerate(maneuver,EVENT_ID)
                    EVENT_ID += 1
            
        elif event == 'change lane to left':
            for maneuver_group in root.findall(".//ManeuverGroup"):
                for maneuver in maneuver_group.findall("Maneuver"):
                    write_xosc_change_lane(maneuver,'1',EVENT_ID)
                    EVENT_ID += 1
                    write_xosc_accelerate(maneuver,EVENT_ID)
                    EVENT_ID += 1
        elif event == 'constant speed':
            for maneuver_group in root.findall(".//ManeuverGroup"):
                for maneuver in maneuver_group.findall("Maneuver"):
                    write_xosc_const(maneuver,EVENT_ID)
                    EVENT_ID += 1
        elif event == 'stop': #有问题！！！
            for maneuver_group in root.findall(".//ManeuverGroup"):
                for maneuver in maneuver_group.findall("Maneuver"):
                    write_xosc_stop(maneuver,EVENT_ID)
                    EVENT_ID += 1
        elif event == 'route navigation':
            for maneuver_group in root.findall(".//ManeuverGroup"):
                for maneuver in maneuver_group.findall("Maneuver"):
                    write_xosc_navigation(maneuver,EVENT_ID,TargetLocation_npc)
                    EVENT_ID += 1
        elif event == 'hazard decelerate':
            for maneuver_group in root.findall(".//ManeuverGroup"):
                for maneuver in maneuver_group.findall("Maneuver"):
                    write_xosc_harzard_dec(maneuver,EVENT_ID,TargetLocation_npc)
                    EVENT_ID += 1
            

In [5]:
# 读取.xosc 模板文件
if 'car' in actors_["npc_actor"]["type"] or 'emergency' in actors_["npc_actor"]["type"] \
    or 'car with left open door' in actors_["npc_actor"]["type"]: 
    tree = ET.parse('./TemplateOpenScenario/template_car.xosc')
    EGO,NPC = "hero","adversary"
elif 'bike' in actors_["npc_actor"]["type"]:
    tree = ET.parse('./TemplateOpenScenario/template_bike.xosc')
    EGO,NPC = "hero","adversary"
if 'right lane with static traffic' in actors_["npc_actor"]["initial_lane_relative_to_ego"] or\
    'right lane with static traffic' in actors_["ego_vehicle"]["lane_relation"]:
    tree = ET.parse('./TemplateOpenScenario/template_right_static_traffic.xosc')
    EGO,NPC,NPC1,NPC2,NPC3 = "hero","adversary","static_NPC1","static_NPC2","static_NPC3"
if 'pedestrian' in actors_["npc_actor"]["type"]:
    tree = ET.parse('./TemplateOpenScenario/template_pedestrian.xosc')
    EGO,NPC,NPC1,NPC2,NPC3 = "hero","adversary","static_NPC1","static_NPC2","static_NPC3"
try:
    if scenario["temporary changes"]:
        tree = ET.parse('./TemplateOpenScenario/template_barrier.xosc')
        EGO,NPC = "hero","adversary"
except:
    pass
root = tree.getroot()

def change_param(attribute,key,input_param):
    # <attribute key=input_param>
    for logic_file in root.iter(attribute):
        filepath = logic_file.get(key)
        logic_file.set(key, input_param)

def change_param_vehicle_attribute(identifier,input_param):
    for scenario in root.findall(".//ScenarioObject[@name='{}']".format(identifier)):
        vehicle = scenario.find('Vehicle')
        vehicle.attrib['name'] = input_param["name"]
        vehicle.attrib['vehicleCategory'] = input_param["vehicleCategory"]

def change_param_tc_attribute(identifier,input_param):
    for scenario in root.findall(".//ScenarioObject[@name='{}']".format(identifier)):
        vehicle = scenario.find('MiscObject')
        vehicle.attrib['name'] = input_param["name"]

def change_param_location(identifier,input_param):
    # <Private entityRef=identifier>...
    #     <attribute key=input_param...
    for private in root.findall(".//Private[@entityRef='{}']".format(identifier)):
        teleport_action = private.find('PrivateAction/TeleportAction')
        if teleport_action is not None:
            WorldPosition = teleport_action.find('Position/WorldPosition')
            if WorldPosition is not None:
                WorldPosition.attrib['x'] = input_param["x"]
                WorldPosition.attrib['y'] = input_param["y"]
                WorldPosition.attrib['z'] = input_param["z"]
                WorldPosition.attrib['h'] = input_param["h"]

# 基于专家知识匹配查找并替换参数，
# 注意优先级高的应该放在后面，如rainy和nighttime都影响Sun intensity，但是nighttime的优先级更高！！！

# -------------------静态行为----------------------
# weather parameter
if "rainy" in weather_density_:
    change_param('Weather','cloudState',"rainy")
    change_param('Precipitation','precipitationType',"rain")
    change_param('Precipitation','intensity',str(weather_density_.get('rainy')))
    change_param('RoadCondition','frictionScaleFactor',str(-0.2*weather_density_.get('rainy')+1))
    change_param('Sun','intensity',str(-0.8*weather_density_.get('rainy')+1))
elif "sunny" in weather_density_:
    change_param('Weather','cloudState',"free")
    change_param('Precipitation','precipitationType',"dry")
    change_param('Sun','intensity',str(weather_density_.get('sunny')))
    

# time of day
if "nighttime" in time_:
    change_param('TimeOfDay','dateTime',"2020-10-23T22:00:00")
    change_param('Sun','intensity',"0")
    change_param('Sun','azimuth',"0")
    change_param('Sun','elevation',"0")
elif "daytime" in time_:
    change_param('TimeOfDay','dateTime',"2020-10-23T12:00:00")
    change_param('Sun','azimuth',"0")
    change_param('Sun','elevation',"1.31")

TargetLocation_npc = {"x":"","y":"","z":"","h":""}
# initial location match
if 'intersection' in road_type_:
    change_param('LogicFile','filepath',"Town05")
    # 'no matter what traffic sign and road maker is'
    if 'left lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"-47.232","y":"40.941","z":"0.2","h":"270"} #记住需要关闭carla的坐标转换（这是carla的世界坐标）
        change_param_location(EGO,InitialLocation_ego)
        if 'turn left' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"-91.745","y":"-4.441","z":"0.2","h":"180"}
        elif 'go forward' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"-47.232","y":"-46.647","z":"0.2","h":"270"}
        if 'right' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
            if 'right lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"-43.732","y":"40.941","z":"0.2","h":"270"}
                # InitialLocation_npc2 = {"x":"-54.74","y":"-46.647","z":"0.2"}
                # InitialLocation_npc = random.choice([InitialLocation_npc1,InitialLocation_npc2])
                change_param_location(NPC,InitialLocation_npc)
        elif 'front' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
            if 'opposite lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"-51.24","y":"-46.65","z":"0.2","h":"90"}
                change_param_location(NPC,InitialLocation_npc)
            elif 'same lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"-47.232","y":"25.941","z":"0.2","h":"270"}
                change_param_location(NPC,InitialLocation_npc)
            elif 'oncoming from the right at the intersection' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"-3.833","y":"-4.441","z":"0.2","h":"180"}
                change_param_location(NPC,InitialLocation_npc)
            elif 'cross sidewalk' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"-41.732","y":"10","z":"0.2","h":"180"}
                change_param_location(NPC,InitialLocation_npc)
    elif 'right lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"-43.232","y":"40.941","z":"0.2","h":"270"}
        change_param_location(EGO,InitialLocation_ego)
        if 'turn right' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"-3.83","y":"6.273","z":"0.2","h":"360"}
        if 'front' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
            if 'oncoming from the left at the intersection' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"-91.745","y":"6.273","z":"0.2","h":"360"}
                # InitialLocation_npc2 = {"x":"-54.74","y":"-46.647","z":"0.2"}
                # InitialLocation_npc = random.choice([InitialLocation_npc1,InitialLocation_npc2])
                change_param_location(NPC,InitialLocation_npc)

elif 'two-lane road' in road_type_:
    change_param('LogicFile','filepath',"Town05")
    if 'right lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"35.083","y":"71.219","z":"0.2","h":"-90"}
        change_param_location(EGO,InitialLocation_ego)
        if 'turn left' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"31.083","y":"-10.219","z":"0.2","h":"-90"}
        if 'front' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
            if 'same lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"35.083","y":"65.219","z":"0.2","h":"-90"}
                change_param_location(NPC,InitialLocation_npc)
    elif 'left lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"31.383","y":"71.219","z":"0.2","h":"-90"}
        change_param_location(EGO,InitialLocation_ego)
        if 'go forward' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"31.383","y":"-10.219","z":"0.2","h":"-90"}
        if 'front' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
            if 'right lane with static traffic' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                if 'pedestrian' in actors_["npc_actor"]["type"]:
                    InitialLocation_npc= {"x":"35.083","y":"48.219","z":"0.2","h":"-180"}
                else:
                    InitialLocation_npc= {"x":"35.083","y":"44.219","z":"0.2","h":"-90"}
                change_param_location(NPC,InitialLocation_npc)
                InitialLocation_npc1= {"x":"35.083","y":"60.219","z":"0.2","h":"-90"}
                change_param_location(NPC1,InitialLocation_npc1)
                InitialLocation_npc2= {"x":"35.083","y":"52.219","z":"0.2","h":"-90"}
                change_param_location(NPC2,InitialLocation_npc2)
                InitialLocation_npc3= {"x":"35.083","y":"30.219","z":"0.2","h":"-90"}
                change_param_location(NPC3,InitialLocation_npc3)
            elif 'same lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"31.083","y":"44.219","z":"0.2","h":"-90"}
                change_param_location(NPC,InitialLocation_npc)
            elif 'right lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"35.083","y":"44.219","z":"0.2","h":"-90"}
                change_param_location(NPC,InitialLocation_npc)
            elif 'left lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"31.083","y":"54.219","z":"0.2","h":"-90"}
                change_param_location(NPC,InitialLocation_npc)
    elif 'right lane with static traffic' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"35.383","y":"44.219","z":"0.2","h":"-90"}
        change_param_location(EGO,InitialLocation_ego)
        if 'go forward' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"31.383","y":"-10.219","z":"0.2","h":"-90"}
        if 'behind' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
            if 'left lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                InitialLocation_npc= {"x":"31.383","y":"42.219","z":"0.2","h":"-90"}
                change_param_location(NPC,InitialLocation_npc)
                InitialLocation_npc1= {"x":"35.083","y":"60.219","z":"0.2","h":"-90"}
                change_param_location(NPC1,InitialLocation_npc1)
                InitialLocation_npc2= {"x":"35.083","y":"52.219","z":"0.2","h":"-90"}
                change_param_location(NPC2,InitialLocation_npc2)
                InitialLocation_npc3= {"x":"35.083","y":"30.219","z":"0.2","h":"-90"}
                change_param_location(NPC3,InitialLocation_npc3)

elif 'single-lane road' in road_type_:
    change_param('LogicFile','filepath',"Town05")
    if 'left lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"155.087","y":"72.272","z":"0.2","h":"270"}
        change_param_location(EGO,InitialLocation_ego)
        if 'go forward' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"154.9","y":"-88.2","z":"0.2","h":"270"}
            if 'behind' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
                if 'same lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                    InitialLocation_npc= {"x":"155.087","y":"87.3","z":"0.2","h":"270"}
                    TargetLocation_npc = {"x":"154.9","y":"-88.2","z":"0.2","h":"270"}
                    change_param_location(NPC,InitialLocation_npc)
            elif 'front' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
                if 'same lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                    InitialLocation_npc= {"x":"155.087","y":"40","z":"0.2","h":"270"}
                    change_param_location(NPC,InitialLocation_npc)
            try:
                if 'cone barrel' in temporary_changes_["type"]:
                    tc = "tc"
                    tc_attribute = {"name":"static.prop.constructioncone"}
                    change_param_tc_attribute(tc,tc_attribute)
                    if 'front' in temporary_changes_["position_relative_to_ego"]:
                        InitialLocation_tc = {"x":"155.087","y":"40","z":"0.2","h":"270"}
                        change_param_location(tc,InitialLocation_tc)
            except:
                pass

elif 'single-lane bending' in road_type_:
    change_param('LogicFile','filepath',"Town04")
    if 'left lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"171.854","y":"-311.228","z":"0.2","h":"-180"}
        change_param_location(EGO,InitialLocation_ego)
        if 'go forward' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"61.53","y":"-225.903","z":"0.2","h":"-257.475"}
            if 'front' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
                if 'opposite lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                    InitialLocation_npc= {"x":"64.947","y":"-225.144","z":"0.2","h":"-77.475"}
                    change_param_location(NPC,InitialLocation_npc)

elif 'ramp merging' in road_type_:
    change_param('LogicFile','filepath',"Town04")
    if 'left lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"15.266","y":"-43.969","z":"0.5","h":"-90"}
        change_param_location(EGO,InitialLocation_ego)
        if 'go forward' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"-58.685","y":"5.9","z":"10","h":"-180"}
            if 'behind' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
                if 'left lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                    InitialLocation_npc= {"x":"334.3","y":"10.5","z":"10","h":"-180"}
                    change_param_location(NPC,InitialLocation_npc)

elif 'ramp exit' in road_type_:
    change_param('LogicFile','filepath',"Town04")
    if 'left lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"66.34","y":"16.85","z":"11","h":"-180"}
        change_param_location(EGO,InitialLocation_ego)
        if 'turn right' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"-34.73","y":"-88.62","z":"0.2","h":"38.16"}
            if 'behind' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
                if 'right lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                    InitialLocation_npc= {"x":"61.34","y":"6.35","z":"11","h":"-180"}
                    change_param_location(NPC,InitialLocation_npc)

elif 'elevated road' in road_type_:
    change_param('LogicFile','filepath',"Town04")
    if 'right lane' in actors_["ego_vehicle"]["lane_relation"]:
        InitialLocation_ego = {"x":"204.3","y":"8.3","z":"7","h":"-180"}
        change_param_location(EGO,InitialLocation_ego)
        if 'go forward' in actors_["ego_vehicle"]["behavior"]:
            TargetLocation_ego = {"x":"-58.685","y":"5.9","z":"10","h":"-180"}
            if 'right' in actors_["npc_actor"]["initial_position_relative_to_ego"]:
                if 'right lane' in actors_["npc_actor"]["initial_lane_relative_to_ego"]:
                    InitialLocation_npc= {"x":"131.024","y":"-61.429","z":"7","h":"77.3"}
                    change_param_location(NPC,InitialLocation_npc)

print(actors_["npc_actor"]["oracle_of_npc_actor"])
# -------------------npc 动态行为定义----------------------
events_of_npc_define(actors_["npc_actor"]["oracle_of_npc_actor"],root,TargetLocation_npc)

# ego vehicle attribute
if 'car' in actors_["ego_vehicle"]["type"]:
    ego_vehicle_attribute = {"name":"vehicle.tesla.model3","vehicleCategory":"car"}
    change_param_vehicle_attribute(EGO,ego_vehicle_attribute)

# npc vehicle attribute
if 'car' in actors_["npc_actor"]["type"]:
    npc_vehicle_attribute = {"name":"vehicle.lincoln.mkz_2017","vehicleCategory":"car"}
    change_param_vehicle_attribute(NPC,npc_vehicle_attribute)
elif 'bike' in actors_["npc_actor"]["type"]:
    npc_vehicle_attribute = {"name":"vehicle.diamondback.century","vehicleCategory":"bicycle"}
    change_param_vehicle_attribute(NPC,npc_vehicle_attribute)
elif 'emergency' in actors_["npc_actor"]["type"]:
    npc_vehicle_attribute = {"name":"vehicle.carlamotors.firetruck","vehicleCategory":"car"}
    change_param_vehicle_attribute(NPC,npc_vehicle_attribute)
elif 'car with left open door' in actors_["npc_actor"]["type"]:
    npc_vehicle_attribute = {"name":"vehicle.nissan.patrol_2021","vehicleCategory":"car"}
    change_param_vehicle_attribute(NPC,npc_vehicle_attribute)

# 保存修改后的文件
tree = ET.ElementTree(root)
_indent(root)
tree.write('./scenario_rep/scenario_from_LLM.xosc')
# 保存ego车的终点位置
Route_ego = {"start_location":InitialLocation_ego,"target_location":TargetLocation_ego}
with open("./scenario_rep/target_location_of_ego_vehicle.json", "w") as file:
    json.dump(Route_ego, file)


['accelerate', 'change lane to left', 'overtake', 'stop']
